
# Glue Studio Notebook
You are now running a **Glue Studio** notebook; before you can start using your notebook you *must* start an interactive session. The second cell in this notebook contains all the needed magics to start your session so all you need to do is execute it.

## Available Magics
|          Magic              |   Type       |                                                                        Description                                                                        |
|-----------------------------|--------------|-----------------------------------------------------------------------------------------------------------------------------------------------------------|
| %%configure                 |  Dictionary  |  A json-formatted dictionary consisting of all configuration parameters for a session. Each parameter can be specified here or through individual magics. |
| %profile                    |  String      |  Specify a profile in your aws configuration to use as the credentials provider.                                                                          |
| %iam_role                   |  String      |  Specify an IAM role to execute your session with.                                                                                                        |
| %region                     |  String      |  Specify the AWS region in which to initialize a session                                                                                                  |
| %session_id                 |  String      |  Returns the session ID for the running session.                                                                                                          |
| %connections                |  List        |  Specify a comma separated list of connections to use in the session.                                                                                     |
| %additional_python_modules  |  List        |  Comma separated list of pip packages, s3 paths or private pip arguments.                                                                                 |
| %extra_py_files             |  List        |  Comma separated list of additional Python files from S3.                                                                                                 |
| %extra_jars                 |  List        |  Comma separated list of additional Jars to include in the cluster.                                                                                       |
| %number_of_workers          |  Integer     |  The number of workers of a defined worker_type that are allocated when a job runs. worker_type must be set too.                                          |
| %worker_type                |  String      |  Standard, G.1X, *or* G.2X. number_of_workers must be set too. Default is G.1X                                                                            |
| %glue_version               |  String      |  The version of Glue to be used by this session. Currently, the only valid options are 2.0 and 3.0                                                        |
| %security_config            |  String      |  Define a security configuration to be used with this session.                                                                                            |
| %sql                        |  String      |  Run SQL code. All lines after the initial %%sql magic will be passed as part of the SQL code.                                                            |
| %status                     |              |  Returns the status of the current Glue session including its duration, configuration and executing user / role.                                          |
| %delete_session             |              |  Deletes the current session and kills the cluster. User stops being charged.                                                                             |
| %list_sessions              |              |  Lists all currently running sessions by name and ID.                                                                                                     |

In [ ]:
import sys
from awsglue.transforms import *
from awsglue.utils import getResolvedOptions
from pyspark.context import SparkContext
from awsglue.context import GlueContext
from awsglue.job import Job
  
sc = SparkContext.getOrCreate()
glueContext = GlueContext(sc)
spark = glueContext.spark_session
job = Job(glueContext)

Welcome to the Glue Interactive Sessions Kernel
For more information on available magic commands, please type %help in any new cell.

Please view our Getting Started page to access the most up-to-date information on the Interactive Sessions kernel: https://docs.aws.amazon.com/glue/latest/dg/interactive-sessions.html
It looks like there is a newer version of the kernel available. The latest version is 0.30 and you have 0.29 installed.
Please run `pip install --upgrade aws-glue-sessions` to upgrade your kernel
Authenticating with environment variables and user-defined glue_role_arn: arn:aws:iam::795874274480:role/AWSGlueRole-Access
Attempting to use existing AssumeRole session credentials.
Trying to create a Glue session for the kernel.
Worker Type: G.1X
Number of Workers: 5
Session ID: 687ddd2d-a6b6-4ce0-b0b6-3f86bf7cb665
Applying the following default arguments:
--glue_kernel_version 0.29
--enable-glue-datacatalog true
Waiting for session 687ddd2d-a6b6-4ce0-b0b6-3f86bf7cb665 to get int

### Importing tables from Data Catalog

In [1]:
customer_df = glueContext.create_data_frame.from_catalog(database="raw-data-db", table_name="customer")
order_items_df = glueContext.create_data_frame.from_catalog(database="raw-data-db", table_name="order_items")
order_payments_df = glueContext.create_data_frame.from_catalog(database="raw-data-db", table_name="order_payments")
orders_df = glueContext.create_data_frame.from_catalog(database="raw-data-db", table_name="orders")
products_df = glueContext.create_data_frame.from_catalog(database="raw-data-db", table_name="products")

### Dropping Nulls Rows

In [2]:
customer_df_drop = customer_df.na.drop()
order_items_drop = order_items_df.na.drop()
order_payments_drop = order_payments_df.na.drop()
orders_drop = orders_df.na.drop()
products_drop = products_df.na.drop()

In [5]:
from awsglue.dynamicframe import DynamicFrame

### Changing type from DF to Dynamic Frame for upload

In [6]:
customer_df_dynamic = DynamicFrame.fromDF(customer_df_drop, glueContext, "test_customer")
order_items_df_dynamic = DynamicFrame.fromDF(order_items_drop, glueContext, "test_order_items")
order_payments_df_dynamic = DynamicFrame.fromDF(order_payments_drop, glueContext, "test_order_payments")
orders_df_dynamic = DynamicFrame.fromDF(orders_drop, glueContext, "test_orders_df")
products_df_dynamic = DynamicFrame.fromDF(products_drop, glueContext, "test_products")

### Uploading Dynamic Frames to new S3 Bucket

In [7]:
glueContext.write_dynamic_frame.from_options(customer_df_dynamic,connection_type="s3",connection_options={"path":"s3://cleaned-data-bucket-sjb/customer"}, format='csv')
glueContext.write_dynamic_frame.from_options(order_items_df_dynamic,connection_type="s3",connection_options={"path":"s3://cleaned-data-bucket-sjb/order_items"}, format='csv')
glueContext.write_dynamic_frame.from_options(order_payments_df_dynamic,connection_type="s3",connection_options={"path":"s3://cleaned-data-bucket-sjb/order_payments"}, format='csv')
glueContext.write_dynamic_frame.from_options(orders_df_dynamic,connection_type="s3",connection_options={"path":"s3://cleaned-data-bucket-sjb/orders"}, format='csv')
glueContext.write_dynamic_frame.from_options(products_df_dynamic,connection_type="s3",connection_options={"path":"s3://cleaned-data-bucket-sjb/products"}, format='csv')